<a href="https://colab.research.google.com/github/THESHAMIK/Real-Life-Projects/blob/main/Deep_NN/KHDS_Keras_sklearn_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import packages
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
#from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.optimizers import Adam ##because cannot import name 'Adam' from 'keras.optimizers'
from keras.callbacks import EarlyStopping, ModelCheckpoint
#from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
#from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

In [ ]:
df = pd.read_csv("khds_final_1.csv")
df['Product Sulphur'] = pd.to_numeric(df['Product Sulphur'],errors = 'coerce') ##==> This one works as pd.to_numeric works for convert a list, a series, an array, or a tuple to a numeric datatype
filtered_df = df[df['Product Sulphur'].notnull()]
df.dropna(axis=0)
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 709 entries, 0 to 710
Data columns (total 34 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   LABfeed Sulphur                            709 non-null    float64
 1   T1 : FRESH FEED FLOW TO KHDS KGPerHour     709 non-null    float64
 2   T1 : Reactor inlet temperature DEGC        709 non-null    float64
 3   T1 : Hydrogen make-up flow NM3perHour      709 non-null    float64
 4   T1 : Recycle hydrogen flow NM3perHour      709 non-null    float64
 5   T1 : REACTOR OUTLET Temp DEGC              709 non-null    float64
 6   T1 : REACTOR FEED INLET Pressure KGperCM2  709 non-null    float64
 7   T1 : Cold separator pressure KGperCM2      709 non-null    float64
 8   T1 : KERO FEED DENSITY KGperCM2            709 non-null    float64
 9   T2 : FRESH FEED FLOW TO KHDS KGPerHour     709 non-null    float64
 10  T2 : Reactor inlet tempera

In [ ]:
target_col = "Product Sulphur"
X = filtered_df.loc[:, filtered_df.columns != target_col]
y = filtered_df.loc[:, target_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=2021)

In [ ]:
print("XTRAIN :: ytrain shapes::"+ str(X_train.shape + y_train.shape))

XTRAIN :: ytrain shapes::(567, 33, 567)


In [ ]:
X_train.head()

,LABfeed Sulphur,T1 : FRESH FEED FLOW TO KHDS KGPerHour,T1 : Reactor inlet temperature DEGC,T1 : Hydrogen make-up flow NM3perHour,T1 : Recycle hydrogen flow NM3perHour,T1 : REACTOR OUTLET Temp DEGC,T1 : REACTOR FEED INLET Pressure KGperCM2,T1 : Cold separator pressure KGperCM2,T1 : KERO FEED DENSITY KGperCM2,T2 : FRESH FEED FLOW TO KHDS KGPerHour,T2 : Reactor inlet temperature DEGC,T2 : Hydrogen make-up flow NM3perHour,T2 : Recycle hydrogen flow NM3perHour,T2 : REACTOR OUTLET Temp DEGC,T2 : REACTOR FEED INLET Pressure KGperCM2,T2 : Cold separator pressure KGperCM2,T2 : KERO FEED DENSITY KGperCM2,T3 : FRESH FEED FLOW TO KHDS KGPerHour,T3 : Reactor inlet temperature DEGC,T3 : Hydrogen make-up flow NM3perHour,T3 : Recycle hydrogen flow NM3perHour,T3 : REACTOR OUTLET Temp DEGC,T3 : REACTOR FEED INLET Pressure KGperCM2,T3 : Cold separator pressure KGperCM2,T3 : KERO FEED DENSITY KGperCM2,T4 : FRESH FEED FLOW TO KHDS KGPerHour,T4 : Reactor inlet temperature DEGC,T4 : Hydrogen make-up flow NM3perHour,T4 : Recycle hydrogen flow NM3perHour,T4 : REACTOR OUTLET Temp DEGC,T4 : REACTOR FEED INLET Pressure KGperCM2,T4 : Cold separator pressure KGperCM2,T4 : KERO FEED DENSITY KGperCM2
419,2000.0,86021.90109,295.013961,805.301728,14895.57788,295.551416,22.581979,18.585012,689.166864,85617.90755,295.091414,845.412662,14878.67527,295.836288,22.604003,18.607992,688.805922,86099.48667,294.993230,844.489997,14940.19203,295.796407,22.607123,18.599091,689.164303,86460.01260,295.031797,817.443504,14949.43041,295.922666,22.601837,18.592478,688.712130
642,1700.0,72443.19286,292.066222,752.980982,15886.25013,293.512951,22.153329,18.614115,694.906073,72445.54081,292.151484,804.601129,15839.85134,293.845743,22.155323,18.625997,695.113605,72516.25104,292.124805,769.015451,15865.79212,293.407692,22.130655,18.591639,695.073172,72126.96383,291.817534,769.062447,15889.80355,293.563373,22.152541,18.606115,695.372522
249,2000.0,75523.03776,290.943491,773.181403,14760.76771,292.549527,22.036806,18.592275,701.499601,75131.92813,291.092579,760.499984,14598.56227,292.442571,22.042040,18.613078,702.685225,75257.05573,291.065754,783.945437,14528.38252,292.734528,22.021091,18.590698,702.814941,75335.43776,291.093012,789.142989,14536.62181,292.956879,22.051083,18.606669,702.206163
708,1200.0,60425.38188,270.062989,528.413451,12517.83553,272.306049,21.014648,18.605937,693.293790,60522.48813,269.937830,543.695607,12538.70984,272.363892,21.024867,18.610148,693.428608,60362.94141,270.056110,533.556605,12546.30783,272.405243,21.015765,18.600281,693.736376,60353.13302,270.083814,536.398630,12618.29635,272.570469,21.040742,18.611755,693.730737
335,2500.0,37334.09186,283.631855,465.162410,10073.34530,287.435755,20.057154,18.626658,720.123723,37452.27415,283.548467,483.524894,10144.56602,287.446968,20.061056,18.625387,720.097691,37306.37070,283.656686,467.344424,10192.75400,287.319540,20.052672,18.623301,719.745461,37132.63086,283.715846,478.964552,10192.00311,287.291786,20.048456,18.608933,719.036873


In [ ]:
#Scale data, otherwise model will fail.
#Standardize features by removing the mean and scaling to unit variance
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
type(X_train_scaled)

numpy.ndarray

In [ ]:
type(X_train)

pandas.core.frame.DataFrame

In [ ]:
X_train_scaled.view()

array([[ 0.53407746,  0.9658201 ,  0.37290493, ...,  0.57395306,
         0.10508062,  0.20690118],
       [-0.17561927,  0.38341511,  0.27669157, ...,  0.37972304,
         0.11224147,  0.24259106],
       [ 0.53407746,  0.51551288,  0.24004593, ...,  0.33586308,
         0.11253261,  0.2792093 ],
       ...,
       [ 0.53407746, -0.91307536, -0.03197214, ..., -0.59240925,
         0.12403911,  0.25970944],
       [ 0.53407746,  0.40962512,  0.32385226, ...,  0.37672823,
         0.10402559,  0.3108514 ],
       [-1.35844716, -1.4196743 , -0.61528425, ..., -0.63458177,
         0.060748  ,  0.32228763]])

We need to review this later why some of values r negative? Looks like LAB Feed is negative?

In [ ]:
X_train_scaled.shape

(567, 33)

Hyperparameter tuning for learning rate ==> https://www.youtube.com/watch?v=qk9gUCdb3aw 187.

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
import keras

print(tf.__version__)
print(keras.__version__)

2.8.0
2.8.0


In [ ]:
def define_model_1(learning_rate=0.01):   
    model1 = Sequential()
    model1.add(Dense(64, activation='relu', kernel_initializer='uniform', 
                    input_dim = 33)) 
    model1.add(Dropout(0.1))
    model1.add(Dense(64, kernel_initializer='uniform', activation='relu'))
    model1.add(Dense(1, kernel_initializer='uniform'))
    optimizer = Adam(lr=learning_rate)
    # compile the model
    model1.compile(loss='mse',
                  optimizer=optimizer,      
                  metrics=['mae'])
    return model1

In [ ]:
batch_size = 100
epochs = 10
learning_rate = [0.0001, 0.001, 0.01, 0.1]
model11 = KerasRegressor(build_fn=define_model_1, 
                        epochs=epochs, 
                        batch_size = batch_size, 
                        verbose=1)



In [ ]:
param_grid = dict(learning_rate=learning_rate)
grid = GridSearchCV(estimator=model11, param_grid=param_grid, n_jobs=-1, cv=3)

grid_result = grid.fit(X_train, y_train)


Epoch 1/10
6/6 [==============================] - 1s 3ms/step - loss: 364014.0938 - mae: 320.1018
Epoch 2/10
6/6 [==============================] - 0s 3ms/step - loss: 2493.9902 - mae: 43.2060
Epoch 3/10
6/6 [==============================] - 0s 3ms/step - loss: 766.6806 - mae: 22.3299
Epoch 4/10
6/6 [==============================] - 0s 3ms/step - loss: 368.9601 - mae: 16.6321
Epoch 5/10
6/6 [==============================] - 0s 3ms/step - loss: 142.6519 - mae: 9.9654
Epoch 6/10
6/6 [==============================] - 0s 3ms/step - loss: 99.9079 - mae: 8.3649
Epoch 7/10
6/6 [==============================] - 0s 3ms/step - loss: 45.5371 - mae: 5.0992
Epoch 8/10
6/6 [==============================] - 0s 3ms/step - loss: 39.1110 - mae: 4.8581
Epoch 9/10
6/6 [==============================] - 0s 3ms/step - loss: 24.0254 - mae: 3.4475
Epoch 10/10
6/6 [==============================] - 0s 3ms/step - loss: 20.1046 - mae: 2.9403


In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -24.619279 using {'learning_rate': 0.01}


In [ ]:
grid_result = grid.fit(X_train_scaled, y_train)

Epoch 1/10
6/6 [==============================] - 0s 4ms/step - loss: 26.2990 - mae: 3.6740
Epoch 2/10
6/6 [==============================] - 0s 3ms/step - loss: 17.3185 - mae: 2.4918
Epoch 3/10
6/6 [==============================] - 0s 3ms/step - loss: 15.3877 - mae: 1.9478
Epoch 4/10
6/6 [==============================] - 0s 4ms/step - loss: 14.0390 - mae: 1.9740
Epoch 5/10
6/6 [==============================] - 0s 3ms/step - loss: 13.4874 - mae: 2.0425
Epoch 6/10
6/6 [==============================] - 0s 3ms/step - loss: 12.5785 - mae: 1.8933
Epoch 7/10
6/6 [==============================] - 0s 3ms/step - loss: 12.0447 - mae: 1.8312
Epoch 8/10
6/6 [==============================] - 0s 3ms/step - loss: 11.7117 - mae: 1.7944
Epoch 9/10
6/6 [==============================] - 0s 3ms/step - loss: 11.3684 - mae: 1.8973
Epoch 10/10
6/6 [==============================] - 0s 3ms/step - loss: 10.9890 - mae: 1.7312


In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -11.919380 using {'learning_rate': 0.01}


SO LEARNING rate is freezed to 0.01 whether data scaled or not.

loss: 20.1046 - mae: 2.9403 for UNSCALED data.
loss: 10.9890 - mae: 1.7312 for SCALED data.

In [ ]:
def define_model_2(activation='relu', init_weights='uniform', optimizer='Adam'):   
    model2 = Sequential()
    model2.add(Dense(64, activation=activation, kernel_initializer=init_weights, 
                    input_dim = 33)) 
    model2.add(Dropout(0.1))
    model2.add(Dense(64, kernel_initializer=init_weights, activation=activation))
    model2.add(Dense(1, kernel_initializer=init_weights))
    
    # compile the model
    model2.compile(loss='mean_squared_error',
                  optimizer=optimizer,      
                  metrics=['mae'])
    return model2

https://github.com/bnsreenu/python_for_microscopists/blob/master/188-gridsearch_hyperparam_tuning_activation_opt_weights.py 

In [ ]:
batch_size = 100
epochs = 10

model22 = KerasRegressor(build_fn=define_model_2, 
                        epochs=epochs, 
                        batch_size = batch_size, 
                        verbose=1)


In [ ]:
activation = ['tanh', 'relu', 'sigmoid']
#Also try softplus, tanh, linear, hard_sigmoid 
init_weights = ['uniform', 'normal', 'he_uniform']
#Also try lecun_uniform, he_normal, glorot_normal, etc. 
optimizer = ['SGD', 'RMSprop', 'Adam']

param_grid = dict(activation=activation, init_weights=init_weights, optimizer=optimizer)

#n_jobs=16 uses 16 CPUs. Try not to do -1 on your system as it may hang!!!
# -1 refers to using all available CPUs
#Cross validation, cv=3
grid22 = GridSearchCV(estimator=model22, param_grid=param_grid, n_jobs=-1, cv=3)

In [ ]:
grid_result22 = grid22.fit(X_train, y_train)

Epoch 1/10
6/6 [==============================] - 0s 3ms/step - loss: 15.2126 - mae: 2.4931
Epoch 2/10
6/6 [==============================] - 0s 3ms/step - loss: 12.6972 - mae: 2.0434
Epoch 3/10
6/6 [==============================] - 0s 3ms/step - loss: 13.3538 - mae: 2.2473
Epoch 4/10
6/6 [==============================] - 0s 4ms/step - loss: 12.8191 - mae: 2.0705
Epoch 5/10
6/6 [==============================] - 0s 4ms/step - loss: 13.1938 - mae: 2.1557
Epoch 6/10
6/6 [==============================] - 0s 3ms/step - loss: 12.5246 - mae: 2.0466
Epoch 7/10
6/6 [==============================] - 0s 3ms/step - loss: 12.4706 - mae: 2.0684
Epoch 8/10
6/6 [==============================] - 0s 3ms/step - loss: 12.6740 - mae: 2.0517
Epoch 9/10
6/6 [==============================] - 0s 3ms/step - loss: 12.7780 - mae: 2.0711
Epoch 10/10
6/6 [==============================] - 0s 3ms/step - loss: 12.5696 - mae: 2.0484


In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result22.best_score_, grid_result22.best_params_))

Best: -12.269863 using {'activation': 'tanh', 'init_weights': 'he_uniform', 'optimizer': 'SGD'}


In [ ]:
grid_result22_s = grid22.fit(X_train_scaled, y_train)

Epoch 1/10
6/6 [==============================] - 1s 3ms/step - loss: 24.6507 - mae: 3.4538
Epoch 2/10
6/6 [==============================] - 0s 3ms/step - loss: 16.2426 - mae: 2.2368
Epoch 3/10
6/6 [==============================] - 0s 3ms/step - loss: 12.9254 - mae: 1.8801
Epoch 4/10
6/6 [==============================] - 0s 2ms/step - loss: 12.1461 - mae: 1.8745
Epoch 5/10
6/6 [==============================] - 0s 3ms/step - loss: 11.6375 - mae: 1.8200
Epoch 6/10
6/6 [==============================] - 0s 3ms/step - loss: 11.2114 - mae: 1.8085
Epoch 7/10
6/6 [==============================] - 0s 3ms/step - loss: 10.9633 - mae: 1.8997
Epoch 8/10
6/6 [==============================] - 0s 3ms/step - loss: 10.4907 - mae: 1.7699
Epoch 9/10
6/6 [==============================] - 0s 3ms/step - loss: 10.3919 - mae: 1.7471
Epoch 10/10
6/6 [==============================] - 0s 3ms/step - loss: 10.5053 - mae: 1.7871


In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result22_s.best_score_, grid_result22_s.best_params_))

Best: -11.405379 using {'activation': 'tanh', 'init_weights': 'he_uniform', 'optimizer': 'SGD'}


In [ ]:
batch_size = 10
epochs = 200

model22_m = KerasRegressor(build_fn=define_model_2, 
                        epochs=epochs, 
                        batch_size = batch_size, 
                        verbose=1)
grid22_m = GridSearchCV(estimator=model22_m, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result22_m = grid22_m.fit(X_train, y_train)

Epoch 1/200
57/57 [==============================] - 1s 2ms/step - loss: 18.4032 - mae: 2.6132
Epoch 2/200
57/57 [==============================] - 0s 2ms/step - loss: 12.9287 - mae: 2.1007
Epoch 3/200
57/57 [==============================] - 0s 2ms/step - loss: 12.9203 - mae: 2.0855
Epoch 4/200
57/57 [==============================] - 0s 2ms/step - loss: 12.8468 - mae: 2.0676
Epoch 5/200
57/57 [==============================] - 0s 2ms/step - loss: 12.8719 - mae: 2.1336
Epoch 6/200
57/57 [==============================] - 0s 2ms/step - loss: 12.8719 - mae: 2.0843
Epoch 7/200
57/57 [==============================] - 0s 2ms/step - loss: 12.9798 - mae: 2.0827
Epoch 8/200
57/57 [==============================] - 0s 2ms/step - loss: 12.8430 - mae: 2.0983
Epoch 9/200
57/57 [==============================] - 0s 2ms/step - loss: 12.8745 - mae: 2.0568
Epoch 10/200
57/57 [==============================] - 0s 2ms/step - loss: 12.9091 - mae: 2.1224
Epoch 11/200
57/57 [=============================

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result22_m.best_score_, grid_result22_m.best_params_))

Best: -12.024594 using {'activation': 'sigmoid', 'init_weights': 'he_uniform', 'optimizer': 'Adam'}


When batch = 100, epoch = 10 this was result:
Best: -11.405379 using {'activation': 'tanh', 'init_weights': 'he_uniform', 'optimizer': 'SGD'} 
This changed to below when batch = 10 & epoch = 200 (BUT it took huge time::
{'activation': 'sigmoid', 'init_weights': 'he_uniform', 'optimizer': 'Adam'} 
Still NOT getting relu? We got Adam after increasin epoch though.
he_uniform was constant in both cases.

Ok then let's try to FIX the ideal batch & epoch first & will then come back!
The run for best batch_size/epochs/neurons/dropout_rate etc took 2 hrs run. The scores are::
Best: -76.909940 using {'batch_size': 10, 'dropout_rate': 0.2, 'epochs': 200, 'neurons': 10, 'weight_constraint': 3}
Batch_size again as 10. epochs = 200 - All with GridSearch only.


**Hyperparameter tuning for dropout, # neurons, batch size, # epochs, and weight** constraint ==> https://www.youtube.com/watch?v=Bzsxq1JJbbo

https://github.com/bnsreenu/python_for_microscopists/blob/master/189-gridsearch_hyperparam_tuning_dropout_wt_constr_hidden_layer_neurons.py

Grid search hyperparameters - Tuning dropout and weight constraint
A weight constraint is an update to the network that checks the size of the weights. 
If the size exceeds a predefined limit, the weights are rescaled to 
size below the limit.
The maximum norm (maxnorm) is less aggressive than other norms, so preferable.
Use GridSearchCV class from scikit-learn
Keras models can be used in scikit-learn by wrapping them with 
the KerasClassifier or KerasRegressor class.
In GridSearchCV:
n_jobs = -1 --> uses multiple cores (parallelized)
n_jobs = 1 --> do not parallelize (do this only if you get an error with -1)

In [ ]:
from keras.constraints import maxnorm
def define_model_3(dropout_rate=0.0, weight_constraint=0, neurons=10):   
    model3 = Sequential()
    model3.add(Dense(neurons, activation='relu', kernel_initializer='he_uniform', 
                    input_dim = 33, kernel_constraint=maxnorm(weight_constraint))) 
    model3.add(Dropout(dropout_rate))
    model3.add(Dense(neurons, kernel_initializer='he_uniform', activation='relu'))
    model3.add(Dense(1, kernel_initializer='he_uniform'))
    
    # compile the model
    model3.compile(loss='mean_squared_error',
                  optimizer='Adam',      
                  metrics=['mae'])
    return model3

In [ ]:
model33 = KerasRegressor(build_fn=define_model_3, 
                        verbose=1)

In [ ]:
dropout_rate = [0.0, 0.2, 0.4]
weight_constraint = [1, 2, 3]
neurons = [10, 32, 64]
#batch_size = [100, 200, 400]
#epochs = [1, 5, 10]
#Changing batch/epoch & check result.
batch_size = [10, 20, 50]
epochs = [50, 100,200]

In [ ]:
param_grid33 = dict(dropout_rate=dropout_rate, 
                  weight_constraint=weight_constraint,
                  neurons=neurons, batch_size=batch_size, 
                  epochs=epochs)

#n_jobs=16 uses 16 CPUs. Try not to do -1 on your system as it may hang!!!
# -1 refers to using all available CPUs
#Cross validation, cv=3
grid33 = GridSearchCV(estimator=model33, param_grid=param_grid33, n_jobs=-1, cv=5)

grid_result33 = grid33.fit(X_train, y_train)

Epoch 1/10
6/6 [==============================] - 0s 3ms/step - loss: 108534232.0000 - mae: 9234.1924
Epoch 2/10
6/6 [==============================] - 0s 3ms/step - loss: 13481461.0000 - mae: 3401.7466
Epoch 3/10
6/6 [==============================] - 0s 3ms/step - loss: 773833.0625 - mae: 681.7130
Epoch 4/10
6/6 [==============================] - 0s 3ms/step - loss: 2147355.2500 - mae: 1284.0570
Epoch 5/10
6/6 [==============================] - 0s 3ms/step - loss: 2961978.7500 - mae: 1563.4811
Epoch 6/10
6/6 [==============================] - 0s 3ms/step - loss: 1252490.1250 - mae: 970.7170
Epoch 7/10
6/6 [==============================] - 0s 4ms/step - loss: 204641.5469 - mae: 312.2948
Epoch 8/10
6/6 [==============================] - 0s 4ms/step - loss: 278559.5000 - mae: 449.1649
Epoch 9/10
6/6 [==============================] - 0s 3ms/step - loss: 333957.7500 - mae: 529.8441
Epoch 10/10
6/6 [==============================] - 0s 3ms/step - loss: 145621.3125 - mae: 299.8834


In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result33.best_score_, grid_result33.best_params_))


Best: -445778.532813 using {'batch_size': 100, 'dropout_rate': 0.0, 'epochs': 10, 'neurons': 32, 'weight_constraint': 1}


In [ ]:
param_grid33_m = dict(dropout_rate=dropout_rate, 
                  weight_constraint=weight_constraint,
                  neurons=neurons, batch_size=batch_size, 
                  epochs=epochs)

#n_jobs=16 uses 16 CPUs. Try not to do -1 on your system as it may hang!!!
# -1 refers to using all available CPUs
#Cross validation, cv=3
grid33_m = GridSearchCV(estimator=model33, param_grid=param_grid33_m, n_jobs=-1, cv=5)

grid_result33_m = grid33_m.fit(X_train, y_train)

Epoch 1/200
57/57 [==============================] - 1s 2ms/step - loss: 837087232.0000 - mae: 22826.3672
Epoch 2/200
57/57 [==============================] - 0s 2ms/step - loss: 264732000.0000 - mae: 12269.7168
Epoch 3/200
57/57 [==============================] - 0s 2ms/step - loss: 124645976.0000 - mae: 8565.5850
Epoch 4/200
57/57 [==============================] - 0s 2ms/step - loss: 95610552.0000 - mae: 7367.6816
Epoch 5/200
57/57 [==============================] - 0s 2ms/step - loss: 77455040.0000 - mae: 6555.2598
Epoch 6/200
57/57 [==============================] - 0s 2ms/step - loss: 60624636.0000 - mae: 5641.5991
Epoch 7/200
57/57 [==============================] - 0s 2ms/step - loss: 38793072.0000 - mae: 4506.3301
Epoch 8/200
57/57 [==============================] - 0s 2ms/step - loss: 34954500.0000 - mae: 4314.6743
Epoch 9/200
57/57 [==============================] - 0s 2ms/step - loss: 25916632.0000 - mae: 3632.3167
Epoch 10/200
57/57 [==============================] - 0s 2m

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result33_m.best_score_, grid_result33_m.best_params_))

Best: -76.909940 using {'batch_size': 10, 'dropout_rate': 0.2, 'epochs': 200, 'neurons': 10, 'weight_constraint': 3}


Earlier with :
batch_size = [100, 200, 400]
epochs = [1, 5, 10]
result for batch/epoc was ::
Best: -445778.532813 using {'batch_size': 100, 'dropout_rate': 0.0, 'epochs': 10, 'neurons': 32, 'weight_constraint': 1}

Now after batch increase & epoch ::
batch_size = [10, 20, 50]
epochs = [50, 100,200]   SCORES are ::
Best: -76.909940 using {'batch_size': 10, 'dropout_rate': 0.2, 'epochs': 200, 'neurons': 10, 'weight_constraint': 3}



